In [1]:
from ner.llama_NER import Llama2_NER_Base, Llama2_NER_FewShots_Entity, Llama2_NER_FewShots_Random, Llama2_NER_FewShots_Sentence, Llama2_NER
from ner.utils import Utils
from ner.Datasets.Conll2003Dataset import Conll2003Dataset

from mistralai.mistralai import get_mistral_ai_model

# from llm.llama import llm

import pickle 

%load_ext autoreload
%autoreload 2

In [2]:
utils = Utils()
data_te = Conll2003Dataset(utils, split ='test')

[nltk_data] Downloading package stopwords to /home/ubuntu/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
Some weights of the model checkpoint at dslim/bert-base-NER were not used when initializing BertForTokenClassification: ['bert.pooler.dense.weight', 'bert.pooler.dense.bias']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Map:   0%|          | 0/3453 [00:00<?, ? examples/s]

Map:   0%|          | 0/3453 [00:00<?, ? examples/s]

Map:   0%|          | 0/3453 [00:00<?, ? examples/s]

In [5]:
import random 
import pickle
from ner.process_results import get_metrics_all
models : dict[Llama2_NER]= {}
with_verify = False
nb_few_shots = 5
prompt_type = "discussion"
scores = {}
results = {}
quant = ['Q5_0'] # ['Q2_K', 'Q4_0', 'Q5_0', 'Q8_0']
for q in quant: 
    scores[q] = []
    results[q] = []
    llm = get_mistral_ai_model(f"./mistralai/models/finetuned/Mistral-7B-v0.1-mistralai-finetune-2000-@@##/mistral-finetuned-2k-@@##-{q}-v0.gguf")
    for i in range(5):
        r = random.randint(0, 124589)
        data_te_tr, data_te_te = data_te.train_test_split(test_size = 50, seed = r) 
        data_te_tr.select(range(2762))
        model = Llama2_NER_FewShots_Sentence(llm, utils, data_te_tr, with_verify=with_verify, nb_few_shots=nb_few_shots, prompt_type=prompt_type)  
        result = model.invoke_mulitple(data_te_te['text'])
        results[q].append({'values' : result, 'gold' : data_te_te['spans']})
        cm,f1, precision, recall, y_true, y_pred, all_nes = get_metrics_all(result, data_te_te['spans'])
        scores[q].append(f1)
        print(scores)
        del data_te_tr
        del data_te_te
        del model
    with open("./ner/saves/results/results_pred_labels_colnn2003_mistralai_quantization_2000_@@##_50_2762_False_5_discussion.pkl", 'wb') as f :
        pickle.dump(scores,f)

llama_model_loader: loaded meta data with 20 key-value pairs and 291 tensors from ./mistralai/models/finetuned/Mistral-7B-v0.1-mistralai-finetune-2000-@@##/mistral-finetuned-2k-@@##-Q5_0-v0.gguf (version unknown)
llama_model_loader: - tensor    0:                token_embd.weight q5_0     [  4096, 32000,     1,     1 ]
llama_model_loader: - tensor    1:              blk.0.attn_q.weight q5_0     [  4096,  4096,     1,     1 ]
llama_model_loader: - tensor    2:              blk.0.attn_k.weight q5_0     [  4096,  1024,     1,     1 ]
llama_model_loader: - tensor    3:              blk.0.attn_v.weight q5_0     [  4096,  1024,     1,     1 ]
llama_model_loader: - tensor    4:         blk.0.attn_output.weight q5_0     [  4096,  4096,     1,     1 ]
llama_model_loader: - tensor    5:            blk.0.ffn_gate.weight q5_0     [  4096, 14336,     1,     1 ]
llama_model_loader: - tensor    6:              blk.0.ffn_up.weight q5_0     [  4096, 14336,     1,     1 ]
llama_model_loader: - tensor   

{'Q5_0': [0.2631578947368421]}


100%|██████████| 50/50 [03:46<00:00,  4.53s/it]


{'Q5_0': [0.2631578947368421, 0.35714285714285715]}


100%|██████████| 50/50 [04:04<00:00,  4.88s/it]


{'Q5_0': [0.2631578947368421, 0.35714285714285715, 0.35135135135135137]}


100%|██████████| 50/50 [04:11<00:00,  5.02s/it]


{'Q5_0': [0.2631578947368421, 0.35714285714285715, 0.35135135135135137, 0.2247191011235955]}


100%|██████████| 50/50 [04:12<00:00,  5.04s/it]

{'Q5_0': [0.2631578947368421, 0.35714285714285715, 0.35135135135135137, 0.2247191011235955, 0.19318181818181818]}


In [ ]:
with open("./ner/saves/results/results_pred_labels_colnn2003_mistralai_quantization_10k_50_2762_False_5_discussion.pkl", 'wb') as f :
    pickle.dump(scores,f)

In [1]:
scores = {'Q2_K': [0.48, 0.5443037974683544, 0.4489795918367347, 0.5977011494252874, 0.4148936170212766], 'Q4_0': [0.4482758620689655, 0.4434782608695652, 0.47959183673469385, 0.45794392523364486, 0.3670886075949367], 'Q5_0': [0.2204724409448819, 0.17592592592592593, 0.21495327102803738, 0.13934426229508196, 0.16535433070866143], 'Q8_0': [0.22826086956521738, 0.29591836734693877, 0.22857142857142856, 0.25892857142857145, 0.20000000000000004]}


In [7]:
with open(f"./ner/saves/results/results_pred_labels_colnn2003_llama2_quantization_2000_50_2762_False_5_discussion.pkl", 'rb') as f :
    scores = pickle.load(f)

In [9]:
from ner.utils import get_student_conf_interval
for q in scores :
    print(q, get_student_conf_interval(scores[q]), scores[q])

Q2_K Mean :  0.374, 95% Confidence Interval: (0.329, 0.419) [0.3609022556390977, 0.42857142857142855, 0.2672413793103448, 0.47572815533980584, 0.3357664233576642]
Q5_0 Mean :  0.430, 95% Confidence Interval: (0.320, 0.539) [0.3669724770642202, 0.5142857142857142, 0.4074074074074074]
Q8_0 Mean :  0.382, 95% Confidence Interval: (0.353, 0.411) [0.31543624161073824, 0.44776119402985076, 0.37837837837837834, 0.4173228346456692, 0.35135135135135137]
